In [89]:
import pandas as pd
import string
from rapidfuzz import process, fuzz
import time
import io
import time
import numpy as np

# FUZZY MATCH FUNCTION

In [148]:
# build dataframe from api
X_pred = pd.DataFrame({'uniq_id':['ERT34983'],
                       'brand':[None],
                       'model':[None],
                       'title':['BMW R100GS PD'],
                       'price':[4500],
                       'mileage':[5002],
                       'bike_year':[1990],
                       'engine_size':[None]})

In [149]:
X_pred.brand[0] == None

True

In [150]:
def fuzzy_match(X_pred):
    """
    fuction to fuzzy match motorbike name
    """
    #remove punctuation
    def remove_punctuations(text):
        for punctuation in string.punctuation:
            text = text.replace(punctuation, '')
        return text

    #remove punctuation
    if X_pred.brand[0]!=None :
        X_pred.brand = X_pred.brand.apply(remove_punctuations)
        
    if X_pred.model[0]!=None:
        X_pred.model = X_pred.model.apply(remove_punctuations)

    def create_title_is_missing(brand, model, title):
        if title == None:
            return brand + " " + model
        return title

    X_pred.title = X_pred.apply(lambda x: create_title_is_missing(x.brand, x.model, x.title), axis=1)

    #import motorcycle databse
    motorcycle_database = pd.read_csv('../../tresboncoin/data/master_vehicule_list/bikez.csv')

    motorcycle_database.drop(columns=['Unnamed: 0', 'model_inv_db', 'model_submodel_inv_db',
           'engine_type_db', 'torque_db',
           'compression_db', 'cooling_system_db', 'dry_weight_db',
           'power/weight_ratio_db', 'model_size_db', 'model_size_inv_db'], inplace=True)

    def concat(brand, submodel):
        return str(brand) + " " + str(submodel)

    motorcycle_database['brand_submodel_db'] = motorcycle_database.apply(lambda x: concat(x.brand_db, x.model_submodel_db), axis=1)

    def choices(year):
        choices = motorcycle_database[motorcycle_database.year_db == year].brand_submodel_db.unique().tolist()
        return [str(x) for  x in choices]

    def match_model(choices, to_match):
        return process.extractOne(to_match, choices)

    X_pred["fuzzy_result"] = X_pred.apply(lambda x: match_model(choices(x.bike_year), x.title), axis=1)

    def unpack_tuple_name(result):
        try:
            return result[0]
        except:
            return np.nan

    def unpack_tuple_score(result):
        try:
            return result[1]
        except:
            return np.nan

    X_pred['fuzzy_score'] = X_pred['fuzzy_result'].apply(unpack_tuple_score)
    X_pred['fuzzy_brand'] = X_pred['fuzzy_result'].apply(unpack_tuple_name)
    X_pred.drop(columns=['fuzzy_result'], inplace=True)
    X_pred.drop(columns=['brand','model','title'], inplace=True)

    X_pred[['brand','model']] = X_pred.fuzzy_brand.apply(lambda x: pd.Series(str(x).split(" ")))

    X_pred.drop(columns=['fuzzy_brand'], inplace=True)


    X_pred = X_pred.merge(motorcycle_database,how='left', left_on=['brand', 'model', 'bike_year'], right_on=['brand_db', 'model_submodel_db', 'year_db'])
    X_pred.drop(columns=['brand_db','model_db','model_submodel_db','year_db', 'brand_submodel_db', 'fuzzy_score'], inplace=True)

    return X_pred

In [151]:
fuzzy_match(X_pred)

,uniq_id,price,mileage,bike_year,engine_size,brand,model,category_db,engine_size_db,power_db
0,ERT34983,4500,5002,1990,None,bmw,r100gs,touring,979.0,60.0
